In [1]:
import numpy as np 
import pandas as pd 

Mydata = pd.read_csv('../DataOut/bus_trip_all_points.csv')
AyyasData=pd.read_csv('../DataOut/processed_data/bus_running_times_feature_added_all.csv')

In [2]:
# drop raws where only trip_id is null
AyyasData.dropna(subset=['trip_id'], inplace=True)

In [3]:
# show all the columns in pandas dataframes
pd.set_option('display.max_columns', None)

In [4]:
AyyasData.head()


,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,day_of_week,time_of_day,Sunday/holiday,saturday,weekday/end,week_no,rt(w-1),rt(w-2),rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,4.0,6.50,0.0,0,1,1.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,4.0,6.50,0.0,0,1,1.0,247.0,247.0,247.0,247.0,247.0,69.0,247.0,247.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,74.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,4.0,6.75,0.0,0,1,1.0,506.0,506.0,506.0,506.0,506.0,210.0,69.0,506.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,4.0,6.75,0.0,0,1,1.0,192.0,192.0,192.0,192.0,192.0,496.0,210.0,69.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,6.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,4.0,6.75,0.0,0,1,1.0,114.0,114.0,114.0,114.0,114.0,195.0,496.0,210.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0


In [5]:
# AyyasDataonly keep 'trip_id', 'deviceid', 'direction', 'date', 'start_time', 'end_time' columns
AyyasData_forComp = AyyasData[['trip_id', 'deviceid',  'date', 'start_time', 'end_time']]
# Convert 'start_time' and 'end_time' columns to datetime format ( start tieme and end time only have time in hh:mm:ss format)
# AyyasData_forComp['start_time'] = pd.to_datetime(AyyasData_forComp['start_time'], format='%H:%M:%S').dt.time
# AyyasData_forComp['end_time'] = pd.to_datetime(AyyasData_forComp['end_time'], format='%H:%M:%S').dt.time

# Convert 'start_time' and 'end_time' columns to datetime
AyyasData_forComp['start_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['start_time'])
AyyasData_forComp['end_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['end_time'])
# convert 'date' column to datetime format
AyyasData_forComp['date'] = pd.to_datetime(AyyasData_forComp['date'], format='%Y-%m-%d').dt.date


C:\Users\gw\AppData\Local\Temp\ipykernel_98520\1607620677.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AyyasData_forComp['start_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['start_time'])
C:\Users\gw\AppData\Local\Temp\ipykernel_98520\1607620677.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AyyasData_forComp['end_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['end_time'])
C:\Users\gw\AppData\Local\Temp\ipykernel_98520\1607620677.py:11:

In [6]:
# get a list of all columns in the dataframe
colls=list(AyyasData_forComp.columns)
print(colls)

['trip_id', 'deviceid', 'date', 'start_time', 'end_time']


In [7]:
# # group AyyasData_forComp by trip id, and pick any available value to each column , also create new two column of trip start time and trip end time, trip start time will be the earliest time of each trip on start_time column and the trip end time will be the last time value on end_time column in each trip id
# # these are the column names in the dataset ['trip_id', 'deviceid', 'date', 'start_time', 'end_time']
# AyyasData_forComp=AyyasData_forComp.groupby(['trip_id']).agg({'deviceid':'first','date':'first','start_time':'min','end_time':'max'}).reset_index()

In [8]:


# Group by 'trip_id' and aggregate
AyyasData_forComp = AyyasData_forComp.groupby(['trip_id']).agg(
    deviceid=pd.NamedAgg(column='deviceid', aggfunc='first'),
    date=pd.NamedAgg(column='date', aggfunc='first'),
    start_time=pd.NamedAgg(column='start_time', aggfunc='min'),
    end_time=pd.NamedAgg(column='end_time', aggfunc='max')
).reset_index()

# Resulting DataFrame
print(AyyasData_forComp)


       trip_id  deviceid        date          start_time            end_time
0          1.0     262.0  2021-10-01 2021-10-01 06:39:49 2021-10-01 07:27:32
1          3.0     274.0  2021-10-01 2021-10-01 06:58:03 2021-10-01 07:51:21
2          4.0     123.0  2021-10-01 2021-10-01 07:37:40 2021-10-01 08:15:44
3          8.0     274.0  2021-10-01 2021-10-01 08:40:27 2021-10-01 09:34:49
4          9.0     262.0  2021-10-01 2021-10-01 08:40:37 2021-10-01 09:33:22
...        ...       ...         ...                 ...                 ...
14123  25366.0    1358.0  2022-11-01 2022-11-01 17:00:34 2022-11-01 17:46:45
14124  25367.0    1377.0  2022-11-01 2022-11-01 17:40:45 2022-11-01 18:33:10
14125  25368.0     284.0  2022-11-01 2022-11-01 17:51:04 2022-11-01 18:35:06
14126  25369.0     513.0  2022-11-01 2022-11-01 18:08:04 2022-11-01 18:52:05
14127  25370.0     121.0  2022-11-01 2022-11-01 18:20:26 2022-11-01 19:02:30

[14128 rows x 5 columns]


In [9]:
AyyasData_forComp

,trip_id,deviceid,date,start_time,end_time
0,1.0,262.0,2021-10-01,2021-10-01 06:39:49,2021-10-01 07:27:32
1,3.0,274.0,2021-10-01,2021-10-01 06:58:03,2021-10-01 07:51:21
2,4.0,123.0,2021-10-01,2021-10-01 07:37:40,2021-10-01 08:15:44
3,8.0,274.0,2021-10-01,2021-10-01 08:40:27,2021-10-01 09:34:49
4,9.0,262.0,2021-10-01,2021-10-01 08:40:37,2021-10-01 09:33:22
...,...,...,...,...,...
14123,25366.0,1358.0,2022-11-01,2022-11-01 17:00:34,2022-11-01 17:46:45
14124,25367.0,1377.0,2022-11-01,2022-11-01 17:40:45,2022-11-01 18:33:10
14125,25368.0,284.0,2022-11-01,2022-11-01 17:51:04,2022-11-01 18:35:06
14126,25369.0,513.0,2022-11-01,2022-11-01 18:08:04,2022-11-01 18:52:05


In [10]:
# get a list of all columns in the dataframe
colls=list(Mydata.columns)
print(colls)

['id', 'deviceid', 'devicetime', 'latitude', 'longitude', 'speed', 'date', 'time', 'geometry', 'bus_stop', 'trip_id', 'direction']


In [11]:
Mydata.head()

,id,deviceid,devicetime,latitude,longitude,speed,date,time,geometry,bus_stop,trip_id,direction
0,574733098,116,2021-10-16 08:03:04,7.293092,80.635573,9.17927,2021-10-16,08:03:04,POINT (184746.41595346577 232322.75960448402),BT01,2.0,1
1,574733099,116,2021-10-16 08:03:19,7.293068,80.636430,14.03890,2021-10-16,08:03:19,POINT (184841.00784031872 232320.15459256113),NaN,2.0,1
2,574733100,116,2021-10-16 08:03:34,7.293043,80.637552,10.25920,2021-10-16,08:03:34,POINT (184964.84865244984 232317.35306792473),NaN,2.0,1
3,574733101,116,2021-10-16 08:03:49,7.292725,80.637707,7.55940,2021-10-16,08:03:49,POINT (184981.95255171193 232282.14959255166),NaN,2.0,1
4,574733102,116,2021-10-16 08:04:04,7.292292,80.637663,3.77970,2021-10-16,08:04:04,POINT (184977.15733984314 232234.22458233108),NaN,2.0,1


In [12]:
Mydata_forComp=Mydata[['trip_id', 'deviceid', 'date', 'time']]
# Convert 'start_time' and 'end_time' columns to datetime format ( start tieme and end time only have time in hh:mm:ss format)
Mydata_forComp['time'] = pd.to_datetime(Mydata['date'] + ' ' + Mydata['time'])
# convert 'date' column to datetime format
Mydata_forComp['date'] = pd.to_datetime(Mydata['date'], format='%Y-%m-%d').dt.date


C:\Users\gw\AppData\Local\Temp\ipykernel_98520\3053380414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mydata_forComp['time'] = pd.to_datetime(Mydata['date'] + ' ' + Mydata['time'])
C:\Users\gw\AppData\Local\Temp\ipykernel_98520\3053380414.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mydata_forComp['date'] = pd.to_datetime(Mydata['date'], format='%Y-%m-%d').dt.date


In [13]:
# group Mydata by trip id and pick the middle value of every column , columns are ['trip_id', 'deviceid', 'date', 'time']

# Sort the data by 'trip_id' and 'time'
Mydata_forComp = Mydata_forComp.sort_values(by=['trip_id', 'time'])

# Calculate the number of rows in each 'trip_id' group
group_sizes = Mydata_forComp.groupby('trip_id').size().reset_index(name='group_size')

# Select the middle row from each 'trip_id' group
Mydata_forComp = Mydata_forComp.merge(group_sizes, on='trip_id')
Mydata_forComp['middle_row'] = Mydata_forComp['group_size'] // 2

# Select the middle values for each 'trip_id' group
Mydata_forComp = Mydata_forComp[Mydata_forComp.groupby('trip_id').cumcount() == Mydata_forComp['middle_row']]

# Cleanup: Drop the helper columns
Mydata_forComp.drop(columns=['group_size', 'middle_row'], inplace=True)

# Reset the index
Mydata_forComp.reset_index(drop=True, inplace=True)

# Resulting DataFrame with one middle time value for each 'trip_id' group
Mydata_forComp.head()

,trip_id,deviceid,date,time
0,1.0,116,2021-10-16,2021-10-16 07:30:35
1,2.0,116,2021-10-16,2021-10-16 08:28:27
2,3.0,116,2021-10-16,2021-10-16 11:17:34
3,4.0,116,2021-10-16,2021-10-16 12:50:07
4,5.0,116,2021-10-16,2021-10-16 14:40:02


In [14]:
AyyasData_forComp.head()

,trip_id,deviceid,date,start_time,end_time
0,1.0,262.0,2021-10-01,2021-10-01 06:39:49,2021-10-01 07:27:32
1,3.0,274.0,2021-10-01,2021-10-01 06:58:03,2021-10-01 07:51:21
2,4.0,123.0,2021-10-01,2021-10-01 07:37:40,2021-10-01 08:15:44
3,8.0,274.0,2021-10-01,2021-10-01 08:40:27,2021-10-01 09:34:49
4,9.0,262.0,2021-10-01,2021-10-01 08:40:37,2021-10-01 09:33:22


In [15]:
Mydata_forComp.head()

,trip_id,deviceid,date,time
0,1.0,116,2021-10-16,2021-10-16 07:30:35
1,2.0,116,2021-10-16,2021-10-16 08:28:27
2,3.0,116,2021-10-16,2021-10-16 11:17:34
3,4.0,116,2021-10-16,2021-10-16 12:50:07
4,5.0,116,2021-10-16,2021-10-16 14:40:02


In [16]:
# make AyyasData_forComp deviceid integer without the decimal point
AyyasData_forComp['deviceid'] = AyyasData_forComp['deviceid'].astype(int)

In [17]:
smaller_df, larger_df = AyyasData_forComp, Mydata_forComp

# Create a mapping dictionary based on common bus IDs, date, and time
mapping_dict = {}
for index, row in smaller_df.iterrows():
    matching_rows = larger_df[
        (larger_df['deviceid'] == row['deviceid']) &
        (larger_df['time'] >= row['start_time']) &
        (larger_df['time'] <= row['end_time'])
    ]
    if len(matching_rows) >= 1:
        mapping_dict[row['trip_id']] = matching_rows.iloc[0]['trip_id']

# Update trip IDs in the smaller data frame using the mapping dictionary
smaller_df['trip_id'] = smaller_df['trip_id'].map(mapping_dict)

# Now, the trip IDs in the smaller data frame have been updated to match the larger data frame
print(smaller_df)


       trip_id  deviceid        date          start_time            end_time
0       4890.0       262  2021-10-01 2021-10-01 06:39:49 2021-10-01 07:27:32
1          NaN       274  2021-10-01 2021-10-01 06:58:03 2021-10-01 07:51:21
2       2369.0       123  2021-10-01 2021-10-01 07:37:40 2021-10-01 08:15:44
3       6543.0       274  2021-10-01 2021-10-01 08:40:27 2021-10-01 09:34:49
4       4891.0       262  2021-10-01 2021-10-01 08:40:37 2021-10-01 09:33:22
...        ...       ...         ...                 ...                 ...
14123      NaN      1358  2022-11-01 2022-11-01 17:00:34 2022-11-01 17:46:45
14124      NaN      1377  2022-11-01 2022-11-01 17:40:45 2022-11-01 18:33:10
14125      NaN       284  2022-11-01 2022-11-01 17:51:04 2022-11-01 18:35:06
14126      NaN       513  2022-11-01 2022-11-01 18:08:04 2022-11-01 18:52:05
14127      NaN       121  2022-11-01 2022-11-01 18:20:26 2022-11-01 19:02:30

[14128 rows x 5 columns]


In [18]:
# Count the number of unique key-value pairs in mapping_dict
unique_pairs = len({(key, value) for key, value in mapping_dict.items()})

print("Number of unique key-value pairs in mapping_dict:", unique_pairs)


Number of unique key-value pairs in mapping_dict: 10422


In [19]:
mapping_dict

{1.0: 4890.0,
 4.0: 2369.0,
 8.0: 6543.0,
 9.0: 4891.0,
 10.0: 2370.0,
 14.0: 6544.0,
 15.0: 4892.0,
 16.0: 2371.0,
 20.0: 4893.0,
 21.0: 6545.0,
 22.0: 2372.0,
 27.0: 4894.0,
 30.0: 4895.0,
 35.0: 2374.0,
 38.0: 4896.0,
 39.0: 8002.0,
 44.0: 2375.0,
 45.0: 4897.0,
 46.0: 8003.0,
 51.0: 2376.0,
 52.0: 4898.0,
 53.0: 2377.0,
 55.0: 4899.0,
 58.0: 8005.0,
 61.0: 2378.0,
 62.0: 4900.0,
 63.0: 5912.0,
 64.0: 2379.0,
 66.0: 4901.0,
 67.0: 2380.0,
 72.0: 8008.0,
 76.0: 8009.0,
 78.0: 2381.0,
 81.0: 5915.0,
 82.0: 8010.0,
 84.0: 2382.0,
 88.0: 2383.0,
 91.0: 8012.0,
 93.0: 2384.0,
 95.0: 6548.0,
 97.0: 8013.0,
 98.0: 4902.0,
 99.0: 6549.0,
 101.0: 8014.0,
 105.0: 2385.0,
 107.0: 4903.0,
 108.0: 2386.0,
 109.0: 6550.0,
 111.0: 4904.0,
 115.0: 8016.0,
 118.0: 2387.0,
 120.0: 4905.0,
 123.0: 2388.0,
 126.0: 4906.0,
 131.0: 4907.0,
 135.0: 14372.0,
 141.0: 2389.0,
 143.0: 9135.0,
 144.0: 14373.0,
 147.0: 14374.0,
 153.0: 9137.0,
 155.0: 2391.0,
 158.0: 14376.0,
 160.0: 2392.0,
 163.0: 14377.0,
 1

In [20]:
smaller_df

,trip_id,deviceid,date,start_time,end_time
0,4890.0,262,2021-10-01,2021-10-01 06:39:49,2021-10-01 07:27:32
1,NaN,274,2021-10-01,2021-10-01 06:58:03,2021-10-01 07:51:21
2,2369.0,123,2021-10-01,2021-10-01 07:37:40,2021-10-01 08:15:44
3,6543.0,274,2021-10-01,2021-10-01 08:40:27,2021-10-01 09:34:49
4,4891.0,262,2021-10-01,2021-10-01 08:40:37,2021-10-01 09:33:22
...,...,...,...,...,...
14123,NaN,1358,2022-11-01,2022-11-01 17:00:34,2022-11-01 17:46:45
14124,NaN,1377,2022-11-01,2022-11-01 17:40:45,2022-11-01 18:33:10
14125,NaN,284,2022-11-01,2022-11-01 17:51:04,2022-11-01 18:35:06
14126,NaN,513,2022-11-01,2022-11-01 18:08:04,2022-11-01 18:52:05


In [21]:
# drop null rows Mydata
smaller_df.dropna(inplace=True)

In [22]:
smaller_df

,trip_id,deviceid,date,start_time,end_time
0,4890.0,262,2021-10-01,2021-10-01 06:39:49,2021-10-01 07:27:32
2,2369.0,123,2021-10-01,2021-10-01 07:37:40,2021-10-01 08:15:44
3,6543.0,274,2021-10-01,2021-10-01 08:40:27,2021-10-01 09:34:49
4,4891.0,262,2021-10-01,2021-10-01 08:40:37,2021-10-01 09:33:22
5,2370.0,123,2021-10-01,2021-10-01 09:11:33,2021-10-01 09:48:30
...,...,...,...,...,...
14059,3461.0,123,2022-10-31,2022-10-31 17:09:47,2022-10-31 17:57:33
14060,4889.0,250,2022-10-31,2022-10-31 17:40:40,2022-10-31 18:34:39
14061,5908.0,262,2022-10-31,2022-10-31 17:50:44,2022-10-31 18:37:03
14062,11208.0,513,2022-10-31,2022-10-31 18:09:39,2022-10-31 18:52:36


In [23]:
# get the number of nan values in each column
AyyasData.isnull().sum()

trip_id                   0
deviceid                  0
direction                 0
segment                   0
date                   2385
start_time             2385
end_time               2385
run_time               2385
run_time_in_seconds    2385
length                    0
day_of_week               0
time_of_day            2385
Sunday/holiday         2385
saturday                  0
weekday/end               0
week_no                   0
rt(w-1)                2385
rt(w-2)                2385
rt(w-3)                2385
rt(t-1)                2385
rt(t-2)                2385
rt(n-1)                1529
rt(n-2)                 771
rt(n-3)                 809
hour_of_day            2385
day                       0
month                     0
temp                   2385
precip                 2385
windspeed              2385
conditions             2385
dt(n-1)                   0
dtype: int64

In [24]:
AyyasData

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,day_of_week,time_of_day,Sunday/holiday,saturday,weekday/end,week_no,rt(w-1),rt(w-2),rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,4.0,6.50,0.0,0,1,1.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,4.0,6.50,0.0,0,1,1.0,247.0,247.0,247.0,247.0,247.0,69.0,247.0,247.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,74.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,4.0,6.75,0.0,0,1,1.0,506.0,506.0,506.0,506.0,506.0,210.0,69.0,506.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,4.0,6.75,0.0,0,1,1.0,192.0,192.0,192.0,192.0,192.0,496.0,210.0,69.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,6.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,4.0,6.75,0.0,0,1,1.0,114.0,114.0,114.0,114.0,114.0,195.0,496.0,210.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203123,25367.0,1377.0,2.0,30.0,2022-11-01,18:17:36,18:24:21,0:06:45,405.0,2.5600,1.0,18.25,0.0,0,1,42.0,422.0,364.0,422.0,422.0,422.0,195.0,150.0,105.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0
203124,25367.0,1377.0,2.0,31.0,2022-11-01,18:24:21,18:25:36,0:01:15,75.0,0.4200,1.0,18.25,0.0,0,1,42.0,60.0,45.0,60.0,83.0,60.0,405.0,195.0,150.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0
203125,25367.0,1377.0,2.0,32.0,2022-11-01,18:25:49,18:28:10,0:02:21,141.0,1.3000,1.0,18.25,0.0,0,1,42.0,198.0,186.0,198.0,182.0,234.0,75.0,405.0,195.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,13.0
203126,25367.0,1377.0,2.0,33.0,2022-11-01,18:28:10,18:31:25,0:03:15,195.0,1.2200,1.0,18.25,0.0,0,1,42.0,174.0,170.0,174.0,202.0,210.0,141.0,75.0,405.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0


In [25]:
AyyasData['trip_id'] = AyyasData['trip_id'].map(mapping_dict)


In [26]:
AyyasData

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,day_of_week,time_of_day,Sunday/holiday,saturday,weekday/end,week_no,rt(w-1),rt(w-2),rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,4890.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,4.0,6.50,0.0,0,1,1.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
1,4890.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,4.0,6.50,0.0,0,1,1.0,247.0,247.0,247.0,247.0,247.0,69.0,247.0,247.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,74.0
2,4890.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,4.0,6.75,0.0,0,1,1.0,506.0,506.0,506.0,506.0,506.0,210.0,69.0,506.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
3,4890.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,4.0,6.75,0.0,0,1,1.0,192.0,192.0,192.0,192.0,192.0,496.0,210.0,69.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,6.0
4,4890.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,4.0,6.75,0.0,0,1,1.0,114.0,114.0,114.0,114.0,114.0,195.0,496.0,210.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203123,NaN,1377.0,2.0,30.0,2022-11-01,18:17:36,18:24:21,0:06:45,405.0,2.5600,1.0,18.25,0.0,0,1,42.0,422.0,364.0,422.0,422.0,422.0,195.0,150.0,105.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0
203124,NaN,1377.0,2.0,31.0,2022-11-01,18:24:21,18:25:36,0:01:15,75.0,0.4200,1.0,18.25,0.0,0,1,42.0,60.0,45.0,60.0,83.0,60.0,405.0,195.0,150.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0
203125,NaN,1377.0,2.0,32.0,2022-11-01,18:25:49,18:28:10,0:02:21,141.0,1.3000,1.0,18.25,0.0,0,1,42.0,198.0,186.0,198.0,182.0,234.0,75.0,405.0,195.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,13.0
203126,NaN,1377.0,2.0,33.0,2022-11-01,18:28:10,18:31:25,0:03:15,195.0,1.2200,1.0,18.25,0.0,0,1,42.0,174.0,170.0,174.0,202.0,210.0,141.0,75.0,405.0,18.0,1.0,11.0,20.5,0.0,8.3,Overcast,0.0


In [27]:
# get the number of nan values in each column
AyyasData.isnull().sum()

trip_id                52358
deviceid                   0
direction                  0
segment                    0
date                    2385
start_time              2385
end_time                2385
run_time                2385
run_time_in_seconds     2385
length                     0
day_of_week                0
time_of_day             2385
Sunday/holiday          2385
saturday                   0
weekday/end                0
week_no                    0
rt(w-1)                 2385
rt(w-2)                 2385
rt(w-3)                 2385
rt(t-1)                 2385
rt(t-2)                 2385
rt(n-1)                 1529
rt(n-2)                  771
rt(n-3)                  809
hour_of_day             2385
day                        0
month                      0
temp                    2385
precip                  2385
windspeed               2385
conditions              2385
dt(n-1)                    0
dtype: int64

In [28]:
# drop raws where only trip_id is null
AyyasData.dropna(subset=['trip_id'], inplace=True)

In [29]:
AyyasData

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,day_of_week,time_of_day,Sunday/holiday,saturday,weekday/end,week_no,rt(w-1),rt(w-2),rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,4890.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,4.0,6.50,0.0,0,1,1.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
1,4890.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,4.0,6.50,0.0,0,1,1.0,247.0,247.0,247.0,247.0,247.0,69.0,247.0,247.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,74.0
2,4890.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,4.0,6.75,0.0,0,1,1.0,506.0,506.0,506.0,506.0,506.0,210.0,69.0,506.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
3,4890.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,4.0,6.75,0.0,0,1,1.0,192.0,192.0,192.0,192.0,192.0,496.0,210.0,69.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,6.0
4,4890.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,4.0,6.75,0.0,0,1,1.0,114.0,114.0,114.0,114.0,114.0,195.0,496.0,210.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202698,3462.0,123.0,2.0,28.0,2022-10-31,18:45:44,18:47:23,0:01:39,99.0,0.8700,0.0,18.75,0.0,0,1,42.0,120.0,120.0,120.0,120.0,120.0,63.0,156.0,197.0,18.0,31.0,10.0,22.2,0.0,4.3,Partially cloudy,9.0
202699,3462.0,123.0,2.0,29.0,2022-10-31,18:47:38,18:50:23,0:02:45,165.0,1.4000,0.0,18.75,0.0,0,1,42.0,218.0,218.0,218.0,218.0,218.0,99.0,63.0,156.0,18.0,31.0,10.0,22.2,0.0,4.3,Partially cloudy,15.0
202700,3462.0,123.0,2.0,30.0,2022-10-31,18:50:56,18:56:33,0:05:37,337.0,2.5600,0.0,18.75,0.0,0,1,42.0,391.0,391.0,391.0,391.0,391.0,165.0,99.0,63.0,18.0,31.0,10.0,22.2,0.0,4.3,Partially cloudy,33.0
202701,3462.0,123.0,2.0,31.0,2022-10-31,18:56:48,18:57:34,0:00:46,46.0,0.4200,0.0,18.75,0.0,0,1,42.0,65.0,65.0,65.0,65.0,65.0,337.0,165.0,99.0,18.0,31.0,10.0,22.2,0.0,4.3,Partially cloudy,15.0


In [30]:
# print number of unique trip_id in Mydata
print(Mydata['trip_id'].nunique())

15891


In [31]:
# get a list of values in mapping_dict dictionary
available_trip_ids=list(mapping_dict.values())

In [32]:
available_trip_ids=list(set(available_trip_ids))

In [33]:
len(available_trip_ids)

10359

In [34]:
# drop the rows in Mydata where trip_id is not in available_trip_ids
Mydata=Mydata[Mydata['trip_id'].isin(available_trip_ids)]

In [35]:
Mydata

,id,deviceid,devicetime,latitude,longitude,speed,date,time,geometry,bus_stop,trip_id,direction
0,574733098,116,2021-10-16 08:03:04,7.293092,80.635573,9.17927,2021-10-16,08:03:04,POINT (184746.41595346577 232322.75960448402),BT01,2.0,1
1,574733099,116,2021-10-16 08:03:19,7.293068,80.636430,14.03890,2021-10-16,08:03:19,POINT (184841.00784031872 232320.15459256113),NaN,2.0,1
2,574733100,116,2021-10-16 08:03:34,7.293043,80.637552,10.25920,2021-10-16,08:03:34,POINT (184964.84865244984 232317.35306792473),NaN,2.0,1
3,574733101,116,2021-10-16 08:03:49,7.292725,80.637707,7.55940,2021-10-16,08:03:49,POINT (184981.95255171193 232282.14959255166),NaN,2.0,1
4,574733102,116,2021-10-16 08:04:04,7.292292,80.637663,3.77970,2021-10-16,08:04:04,POINT (184977.15733984314 232234.22458233108),NaN,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1298712,895005999,1410,2022-02-24 14:51:43,7.291685,80.635282,3.23974,2022-02-24,14:51:43,POINT (184714.16084255013 232167.22405422156),BT01,15560.0,2
1298713,895682420,1410,2022-02-24 17:50:08,7.299123,80.734463,2.69978,2022-02-24,17:50:08,POINT (195665.40886624547 232987.66098395272),BT02,15562.0,2
1298714,895866047,1410,2022-02-24 18:46:53,7.291740,80.635173,10.25920,2022-02-24,18:46:53,POINT (184702.20468358186 232173.30970404382),BT01,15562.0,2
1298725,902155697,1410,2022-02-26 15:10:01,7.299070,80.734537,0.00000,2022-02-26,15:10:01,POINT (195673.50168209182 232981.76628487784),BT02,15574.0,2


In [36]:
# save my data to csv file bus_trip_all_points_moddedDF.csv
Mydata.to_csv('../DataOut/bus_trip_all_points_moddedDF.csv', index=False)

# save AyyasData to csv file bus_running_times_feature_added_all_moddedDF.csv
AyyasData.to_csv('../DataOut/bus_running_times_feature_added_all_moddedDF.csv', index=False)